In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 661.2 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=0955a00a84398c5036878b9e6d63580aa3494a377bc38dcc6baed849a44ee2d2
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2


#Naampadam Dataset


In [2]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# let's now print how the Dataset looks like
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [4]:
raw_datasets.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [5]:
# let's print an instance of dataset
idx=1000
rec=raw_datasets['train'][idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))


सूक्ष्म	0
,	0
लघु	0
एवं	0
मध्यम	0
उद्यम	0
(	0
एमएसएमई	3
)	0
और	0
सड़क	3
परिवहन	4
एवं	4
राजमार्ग	4
मंत्री	0
श्री	0
नितिन	1
गडकरी	2
ने	0
आज	0
जानकारी	0
दी	0
कि	0
उनका	0
मंत्रालय	0
एक	0
कृषि	0
एमएसएमई	0
नीति	0
लाने	0
पर	0
काम	0
कर	0
रहा	0
है	0
जो	0
स्थानीय	0
कच्चे	0
माल	0
का	0
उपयोग	0
करते	0
हुए	0
विनिर्माण	0
उत्पादों	0
के	0
लिए	0
ग्रामीण	0
,	0
जनजातीय	0
,	0
कृषि	0
और	0
वन	0
क्षेत्रों	0
में	0
उद्यमिता	0
विकास	0
पर	0
ध्यान	0
केंद्रित	0
करेगी	0
।	0


In [6]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [7]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [10]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


##Training an NER Model with the dataset -- Load Pre-trained Model

In [11]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Tokenize all texts and align the labels with them**

In [12]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
train_datasett = raw_datasets["train"]
train_dataset=train_datasett.select(range(20000))

train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

**Create Data Collator, Metrics**

In [14]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [15]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

**Set Training Arguments**

In [17]:
batch_size=8
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    evaluation_strategy = "epoch",
    learning_rate=5e-5)

**Training**

In [18]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
trainer.args

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

In [20]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.319000,0.298102,0.690034,0.665035,0.677304,10213,0.594140,0.418557,0.491127,9786,0.699504,0.614118,0.654036,10568,0.667999,0.568522,0.614259,0.910376
2,0.239200,0.261304,0.720679,0.711152,0.715884,10213,0.582748,0.526058,0.552954,9786,0.734439,0.678842,0.705547,10568,0.682880,0.640724,0.661131,0.919216
3,0.202200,0.259096,0.713632,0.732498,0.722942,10213,0.577398,0.559166,0.568136,9786,0.723214,0.705148,0.714067,10568,0.674233,0.667550,0.670875,0.920830


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [21]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.7229
  eval_LOC_number         =      10213
  eval_LOC_precision      =     0.7136
  eval_LOC_recall         =     0.7325
  eval_ORG_f1             =     0.5681
  eval_ORG_number         =       9786
  eval_ORG_precision      =     0.5774
  eval_ORG_recall         =     0.5592
  eval_PER_f1             =     0.7141
  eval_PER_number         =      10568
  eval_PER_precision      =     0.7232
  eval_PER_recall         =     0.7051
  eval_loss               =     0.2591
  eval_overall_accuracy   =     0.9208
  eval_overall_f1         =     0.6709
  eval_overall_precision  =     0.6742
  eval_overall_recall     =     0.6675
  eval_runtime            = 0:04:10.40
  eval_samples_per_second =     53.754
  eval_steps_per_second   =      3.363


In [22]:
# For answers
from datasets import Dataset
data = {'tokens':
[['इसके', 'अलावा', 'मानेसर-बावल', 'इनवेस्टमेंट', 'रीजन', 'के', 'लिए', 'मास्टर', 'प्लान', 'तैयार', 'किया'], ['महोदय', 'हमारे', 'देश', 'में', 'सारा', 'कामकाज', 'बुल्गारियाई', 'में', 'ही', 'किया', 'जाता', 'है', 'हम', 'वकील', 'सारे', 'दस्तावेज', 'केवल', 'बुल्गारियाई', 'में', 'ही', 'तैयार', 'करते', 'हैं', 'और', 'न्यायालय', 'में', 'भी', 'सभी', 'कार्यवाही', 'बुल्गारियाई', 'में', 'है'], ['भूलने', 'की', 'यह', 'बीमारी', 'अगर', 'हद', 'से', 'ज्यादा', 'बढ़', 'जाए', 'तो', 'यह', 'एक', 'गंभीर', 'रूप', 'धारण', 'कर', 'लेती', 'है'], ['ये', 'भी', 'पढ़ें', 'CAA', 'विरोध-प्रदर्शन', 'पीएम', 'मोदी', 'बोले', '-', 'झूठी', 'अफवाहों', 'में', 'आकर', 'हिंसा', 'ना', 'फैलाएं'], ['महिलाओं', 'के', 'सशक्तिकरण', 'के', 'लिए', 'संसद', 'और', 'विधानसभाओं', 'में', '33', 'प्रतिशत', 'आरक्षण', 'दिए', 'जाने', 'का', 'प्रस्ताव', 'है', '।'], ['मम्मियां', 'उनके', 'मथ्थे', 'पर', 'गिरते', 'हुये', 'बालों', 'को', 'उंगली', 'से', 'बार', 'बार', 'संवारतीं', 'पर', 'वो', 'फिर', 'गिर', 'जाते', '.'], ['इंडेक्स', 'फंड', 'में', 'निवेश', 'करने', 'वालों', 'को', 'ट्रेकिंग', 'एरर', 'से', 'अपने', 'निवेश', 'को', 'सुरक्षित', 'रखने', 'का', 'बेहतर', 'इंतजाम', 'करना', 'चाहिए'], ['पर', 'इस', 'बात', 'को', 'भी', 'दरकिनार', 'नहीं', 'किया', 'जा', 'सकता', 'है', 'कि', 'इन', 'फिल्मों', 'की', 'असफलता', 'का', 'सबसे', 'ज़्यादा', 'नुक़सान', 'उन्हें', 'ही', 'हुआ', '.'], ['इस', 'दौरान', 'वह', 'आज़ादी', 'के', 'नारे', 'भी', 'लगा', 'रही', 'हैं', '।'], ['2', '-', '3', 'किलोमीटर', 'की', 'परिधि', 'में', 'फैले', 'इस', 'छोटे', 'से', 'इलाके', 'ने', 'मलक्का', 'के', '600', 'सालों', 'के', 'इतिहास', 'को', 'समेट', 'रखा', 'है', '.'], ['उन्होंने', 'कहा', 'कि', 'अगर', 'मैं', 'किसी', 'को', 'अपना', 'छात्र', 'बनाता', 'हूं', 'तो', 'उसे', 'अपने', 'बच्चे', 'जैसा', 'मानता', 'हूं', '।'], ['उन्होंने', 'कहा', 'कि', 'अपने', 'गठन', 'के', 'चार', 'दशकों', 'में', 'सोसायटी', 'ने', 'निर्धन', 'और', 'जरूरतमंद', 'लोगों', 'को', 'हरसंभव', 'सहायता', 'प्रदान', 'की', 'है', '।'], ['आज', 'की', 'तारीख', 'है', '10', 'अगस्त', '2018', 'तथा', 'आज', 'इस', 'वर्ष', 'का', '222वां', 'दिन', 'है।', 'इन', '222', 'दिनों', 'में', 'प्रारंग', 'अपने', 'परिवार', 'के', 'जौनपुरवासियों', 'तक', '208', 'लेख', 'पहुंचा', 'चुका', 'है', 'तथा', 'यह', 'लेख', '209वां', 'लेख', 'होगा', '।'], ['उत्तराखंड', 'में', 'चुनाव', 'से', 'पहले', 'नये', 'जिले', 'की', 'चर्चा', 'पर', 'मुख्यमंत्री', 'हरीश', 'रावत', 'ने', 'विराम', 'लगा', 'दिया', 'हैान', 'की', 'है', '.'], ['आप', 'अधिकारियों', 'के', 'नाम,', 'उनके', 'पद,', 'शाखा', 'कार्यालय,', 'प्रोफाइल', 'और', 'फोन', 'नंबर', 'की', 'जानकारी', 'प्राप्त', 'कर', 'सकते', 'हैं', '।'], ['जड़ी', 'बूटी', 'हनीकिल', 'पौधों', 'के', 'लिए', 'honeysuckle', 'और', 'एक', 'ही', 'पौधे', 'सूखे', 'कली', 'या', 'फूलों', 'से', 'संबंधि', 'हैं', '।'], ['रिपोर्ट', 'की', 'माने', 'तो', 'इस', 'पर', 'एक', 'भारतीय', 'अधिकारी', 'ने', 'कहा', 'सरकार', 'विदेशी', 'मोटरसाइकिलों', 'पर', 'आयात', 'शुल्क', 'अधिक', 'लगाती', 'है', 'क्योंकि', 'घरेलू', 'उद्योग', 'को', 'सुरक्षा', 'देने', 'के', 'लिए', 'यह', 'जरूरी', 'है', '.'], ['इस', 'पोस्टर', 'को', 'बस', 'और', 'दीवारों', 'पर', 'चस्पा', 'किया', 'गया', 'हैी', 'है', '।'], ['मेलबर्न', 'खेलों', 'में', 'भारत', 'ने', 'निशोनबाजी', 'में', '16', 'स्वर्ण', 'सहित', '27', 'पदक', 'अपने', 'नाम', 'किए', 'थे'], ['नेताओं', 'के', 'लिए', 'तो', 'हर', 'मौसम', 'ही', 'मुफ़ीद', 'है', 'लेकिन', 'करेंगे', 'वहीं', 'ढाक', 'के', 'तीन', 'पात,', 'इनके', 'यहां', 'तो', 'न', 'सावन', 'सूखे', 'न', 'भादों', 'हरे'], ['#', 'पीएम', 'मोदी', 'ने', 'अपने', 'भाषण', 'में', 'कहा', 'कि', 'हम', 'विकास', 'मानवीयता', 'के', 'साथ', 'चाहते', 'हैं,', '।', 'उन्होंने', 'कहा', 'कि', 'हम', 'देश', 'का', 'वैभव', 'चाहते', 'हैं', 'लेकिन', 'सादगी', 'के', 'धरातल', 'पर'], ['यह', 'वीडियो', '2014', 'के', 'चुनाव', 'से', 'पहले', '2013', 'में', 'पटना', 'की', 'एक', 'रैली', 'के', 'दौरान', 'विस्फोट', 'में', 'मारे', 'गये', 'एक', 'व्यक्ति', 'की', 'पत्नी', 'का', 'है', '।'], ['भीमा-कोरेगांव', 'में', 'दो', 'समुदायों', 'के', 'बीच', 'कल', 'शाम', 'हुई', 'हिंसा', 'में', 'एक', 'व्यक्ति', 'की', 'मौत', 'हो', 'गयी', 'तथा', 'कई', 'अन्य', 'घायल', 'हुए', 'हैं', '।', 'घायलों', 'को', 'शहर', 'के', 'अलग-अलग', 'इलाकों', 'के', 'निजी', 'अस्पतालों', 'में', 'भर्ती', 'कराया', 'गया', 'है', '।'], ['3', '-', 'आज', 'के', 'समय', 'में', 'लोग', 'थोड़ा', 'वजन', 'उठाने', 'से', 'भी', 'कतराते', 'हैं।', 'पर', 'हम', 'आपकी', 'जानकारी', 'के', 'लिए', 'बताते', 'चलें', 'कि', 'आपकी', 'यह', 'गलती', 'आपकी', 'पीठ', 'दर्द', 'का', 'कारण', 'बन', 'सकती', 'हैं', '।'], ['बैंक', 'ने', 'बताया', 'कि', 'मौजूदा', 'वक्त', 'में', 'उसके', '42', 'करोड़', 'सेविंग', 'अकाउंट', 'होल्डर्स', 'हैं', '.']],
'ner_tags':
[[0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
}

dataset = Dataset.from_dict(data)

In [23]:
tokenized_test_set = {}

# for lang in raw_datasets:
tokenized_test_set = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset of language {0}".format('hi'),
)

Running tokenizer on test dataset of language hi #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset of language hi #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
final_metrics = {}

# for lang in tokenized_test_set:
predictions, labels, metrics = trainer.predict(tokenized_test_set, metric_key_prefix=lang)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1'] = metrics[key]
final_metrics = lang_specific_results

/opt/conda/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
print(final_metrics)

{'Precision': 0.38461538461538464, 'Recall': 0.8333333333333334, 'F1': 0.5263157894736842}


In [26]:
model.save_pretrained("ner_model")

In [27]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [28]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}
print(id2label)
print(label2id)

{'0': 'O', '1': 'B-PER', '2': 'I-PER', '3': 'B-ORG', '4': 'I-ORG', '5': 'B-LOC', '6': 'I-LOC'}
{'O': '0', 'B-PER': '1', 'I-PER': '2', 'B-ORG': '3', 'I-ORG': '4', 'B-LOC': '5', 'I-LOC': '6'}


In [29]:
import json
config=json.load(open("/kaggle/working/ner_model/config.json"))

In [30]:
config["id2label"] = id2label
config["label2id"] = label2id

In [31]:
json.dump(config, open("/kaggle/working/ner_model/config.json","w"))

In [32]:
model = AutoModelForTokenClassification.from_pretrained("ner_model")

In [33]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [34]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained("/kaggle/working/output_dir/checkpoint-3500")


In [35]:
# let us try with some example sentences here
# sentence = 'लगातार हमलावर हो रहे शिवपाल और राजभर को सपा की दो टूक, चिट्ठी जारी कर कहा- जहां जाना चाहें जा सकते हैं'
sentences = ['इसके अलावा मानेसर-बावल इनवेस्टमेंट रीजन के लिए मास्टर प्लान  तैयार किया', ' महोदय, हमारे देश में सारा कामकाज बुल्गारियाई में ही किया जाता है हम वकील सारे दस्तावेज केवल बुल्गारियाई में ही तैयार करते हैं और न्यायालय में भी सभी कार्यवाही बुल्गारियाई में होती है."', 'भूलने की यह बीमारी अगर हद से ज्यादा बढ़ जाए तो यह एक गंभीर रूप धारण कर लेती है.', 'ये भी पढ़ें: CAA विरोध-प्रदर्शन: पीएम मोदी बोले- झूठी अफवाहों में आकर हिंसा ना फैलाएं', 'महिलाओं के सशक्तिकरण के लिए संसद और विधानसभाओं में 33 प्रतिशत आरक्षण दिए जाने का प्रस्ताव है।', 'मम्मियां उनके मथ्थे पर गिरते हुये बालों को उंगली से बार बार संवारतीं पर वो फिर गिर जाते.', 'इंडेक्स फंड में निवेश करने वालों को ट्रेकिंग एरर से अपने निवेश को सुरक्षित रखने का बेहतर इंतजाम करना चाहिए।', 'पर इस बात को भी दरकिनार नहीं किया जा सकता है कि इन फिल्मों की असफलता का सबसे ज़्यादा नुक़सान उन्हें ही हुआ.', 'इस दौरान वह आज़ादी के नारे भी लगा रही हैं।', '2-3 किलोमीटर की परिधि में फैले इस छोटे से इलाके ने मलक्का के 600 सालों के इतिहास को समेट रखा है.', 'उन्होंने कहा कि अगर मैं किसी को अपना छात्र बनाता हूं तो उसे अपने बच्चे जैसा मानता हूं।', 'उन्होंने कहा कि अपने गठन के चार दशकों में सोसायटी ने निर्धन और जरूरतमंद लोगों को हरसंभव सहायता प्रदान की है।', 'आज की तारीख है 10 अगस्त 2018 तथा आज इस वर्ष का 222वां दिन है। इन 222 दिनों में प्रारंग अपने परिवार के जौनपुरवासियों तक 208 लेख पहुंचा चुका है तथा यह लेख 209वां लेख होगा।', 'उत्तराखंड में चुनाव से पहले नये जिले की चर्चा पर मुख्यमंत्री हरीश रावत ने विराम लगा दिया है.', 'आप अधिकारियों के नाम, उनके पद, शाखा कार्यालय, प्रोफाइल और फोन नंबर की जानकारी प्राप्त कर सकते हैं।', 'जड़ी बूटी हनीकिल पौधों के लिए honeysuckle और एक ही पौधे सूखे कली या फूलों से संबंधित हैं।', 'रिपोर्ट की माने तो इस पर एक भारतीय अधिकारी ने कहा सरकार विदेशी मोटरसाइकिलों पर आयात शुल्क अधिक लगाती है क्योंकि घरेलू उद्योग को सुरक्षा देने के लिए यह जरूरी है.', 'इस पोस्टर को बस और दीवारों पर चस्पा किया गया है।', 'मेलबर्न खेलों में भारत ने निशोनबाजी में 16 स्वर्ण सहित 27 पदक अपने नाम किए थे।', 'नेताओं के लिए तो हर मौसम ही मुफ़ीद है लेकिन करेंगे वहीं ढाक के तीन पात, इनके यहां तो न सावन सूखे न भादों हरे.', '# पीएम मोदी ने अपने भाषण में कहा कि हम विकास मानवीयता के साथ चाहते हैं। उन्होंने कहा कि हम देश का वैभव चाहते हैं लेकिन सादगी के धरातल पर।', 'यह वीडियो 2014 के चुनाव से पहले 2013 में पटना की एक रैली के दौरान विस्फोट में मारे गये एक व्यक्ति की पत्नी का है।', 'भीमा-कोरेगांव में दो समुदायों के बीच कल शाम हुई हिंसा में एक व्यक्ति की मौत हो गयी तथा कई अन्य घायल हुए हैं। घायलों को शहर के अलग-अलग इलाकों के निजी अस्पतालों में भर्ती कराया गया है।', '3- आज के समय में लोग थोड़ा वजन उठाने से भी कतराते हैं। पर हम आपकी जानकारी के लिए बताते चलें कि आपकी यह गलती आपकी पीठ दर्द का कारण बन सकती हैं।', 'बैंक ने बताया कि मौजूदा वक्त में उसके 42 करोड़ सेविंग अकाउंट होल्डर्स हैं.']

pred_labels = []
for sentence in sentences:
    predicted_labels = get_predictions(sentence=sentence,
                                       tokenizer=tokenizer,
                                       model=model
                                       )
    pred_labels.append(predicted_labels)


In [36]:
print(pred_labels)

[['LABEL_0', 'LABEL_0', 'LABEL_5', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0'], ['LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_5', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_3', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0'], ['LABEL_1', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0'], ['LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_1', 'LABEL_5', 'LABEL_5', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0'], ['LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 'LABEL_0', 

[['B-LOC', 'B-LOC', 'I-PER', 'B-LOC', 'I-PER', 'O', 'O', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'I-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-ORG', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'I-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['I-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'I-PER', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'I-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['I-PER', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-ORG', 'B-PER', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-ORG', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['I-PER', 'B-LOC', 'B-LOC', 'I-PER', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'I-PER', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-ORG', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'I-PER', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['I-PER', 'O', 'O', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC'], ['B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC', 'B-LOC']]

In [37]:
!zip -r ner_model.zip /kaggle/working/ner_model

  adding: kaggle/working/ner_model/ (stored 0%)
  adding: kaggle/working/ner_model/config.json (deflated 51%)
  adding: kaggle/working/ner_model/model.safetensors (deflated 7%)


In [38]:
!zip -r bert_tokenizer.zip /kaggle/working/ner_model

  adding: kaggle/working/ner_model/ (stored 0%)
  adding: kaggle/working/ner_model/config.json (deflated 51%)
  adding: kaggle/working/ner_model/model.safetensors (deflated 7%)


In [38]:
!zip -r ner_model.zip /kaggle/working/

updating: kaggle/working/ner_model/ (stored 0%)
updating: kaggle/working/ner_model/model.safetensors (deflated 7%)
updating: kaggle/working/ner_model/config.json (deflated 51%)
  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/bert_tokenizer.zip (stored 0%)
  adding: kaggle/working/tokenizer/ (stored 0%)
  adding: kaggle/working/tokenizer/special_tokens_map.json (deflated 49%)
  adding: kaggle/working/tokenizer/spiece.model (deflated 60%)
  adding: kaggle/working/tokenizer/tokenizer_config.json (deflated 74%)
  adding: kaggle/working/tokenizer/tokenizer.json (deflated 77%)
  adding: kaggle/working/wandb/ (stored 0%)
  adding: kaggle/working/wandb/debug.log (deflated 77%)
  adding: kaggle/working/wandb/latest-run/ (stored 0%)
  adding: kaggle/working/wandb/latest-run/logs/ (stored 0%)
  adding: kaggle/working/wandb/latest-run/logs/debug.log (deflated 77%)
  adding: kaggle/working/wandb/latest-run/logs/debug-internal.log (deflated 94%)
  adding: kaggle/working/wandb/latest-r